# BUYER'S FUNNEL – Car Selection

## Load processed data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path

today = datetime.now()
formatted_date = today.strftime("%Y%m%d")

input_file = Path.cwd().parent /f"data/processed_csv/processed_listings_{formatted_date}.csv"
output_path = Path.cwd().parent / f"data/eval_csv"
output_path.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(input_file)
print(f"Loaded {len(df)} listings")

In [ ]:
df.head()
# df.describe(include="all")

## Logistics filter

In [ ]:
allowed_zones = ["S", "C"]

df = df[df["zone_code"].isin(allowed_zones)]
print(f"After zone filter: {len(df)}")

## Trust filter

In [ ]:
df["trust_flag"] = np.where(df["cepikVerified"] == True, "TRUSTED", "RISK")
df["trust_penalty"] = np.where(df["cepikVerified"] == True, 0, 0.15)

## Usage sanity

In [ ]:
current_year = datetime.now().year

df["km_per_year"] = df["mileage"] / (df["car_age"] + 1)

df["usage_flag"] = pd.cut(
    df["km_per_year"],
    bins=[0, 18000, 25000, 100000],
    labels=["NORMAL", "HIGH", "EXTREME"]
)

In [ ]:
df = df[df["km_per_year"] <= 30000]
print(f"After usage filter: {len(df)}")


## Core value metrics

In [ ]:
df["price_per_km"] = (df["price_pln"] / df["mileage"]).round(2)
df["price_per_hp"] = (df["price_pln"] / df["engine_power"]).round(0)

In [ ]:
df["ppk_rank"] = df["price_per_km"].rank(pct=True)
df["pphp_rank"] = df["price_per_hp"].rank(pct=True)
df["usage_rank"] = df["km_per_year"].rank(pct=True)
df["year_rank"] = df["year"].rank(pct=True, ascending=False)

## Value Score

In [ ]:
df["value_score"] = (
    df["ppk_rank"] * 0.45 +
    df["pphp_rank"] * 0.25 +
    df["usage_rank"] * 0.20 +
    df["year_rank"] * 0.10 +
    df["trust_penalty"] 
)

# Lower = better

## Shortlist

In [ ]:
# Toggle cell based on CEPIK verification preference
df = df[df['trust_flag'] == 'TRUSTED']
print(f"After trust filter: {len(df)}")

In [ ]:
shortlist = df.sort_values("value_score").head(20).copy()
shortlist.reset_index(drop=True, inplace=True)
shortlist.head()

## Buy / no-buy flag

In [ ]:

shortlist.to_csv(output_path / f"shortlist_{formatted_date}.csv", index=False)

In [ ]:
shortlist["buy_decision"] = "UNDECIDED"
shortlist["decision_reason"] = ""

In [ ]:
# Manually review decisions
for i in shortlist.url.tolist():
    print(i)

In [ ]:
# shortlist.loc[0, ["buy_decision", "decision_reason"]] = ["UNDECIDED", ""]
# shortlist.loc[1, ["buy_decision", "decision_reason"]] = ["UNDECIDED", ""]
# shortlist.loc[2, ["buy_decision", "decision_reason"]] = ["UNDECIDED", ""]
# shortlist.loc[3, ["buy_decision", "decision_reason"]] = ["UNDECIDED", ""]
# shortlist.loc[4, ["buy_decision", "decision_reason"]] = ["UNDECIDED", ""]

## Final action list

In [ ]:
final_5 = shortlist[shortlist["buy_decision"] == "BUY"].head(10).copy()

final_5.reset_index(drop=True, inplace=True)

final_5[[
    "brand", "model", "year", "price_pln", "mileage",
    "price_per_km", "km_per_year",
    "trust_flag", "seller_name", "url"
]]
